In [1]:
# Load all the data
import xarray as xr
import numpy as np
root = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/src/MODIS/'
f = 'aquaDay_errorGTE03K_04km_2018-01-01.tif'

da = xr.open_rasterio(root+f) #load the temperature data array

ds_MODIS = da.squeeze('band').to_dataset(name='LST') #Make it a dataset


# Method 1

In [14]:

import pandas as pd


def method_one(ds,reference_time):
    """ Calculate the absolute UTC using reference_time as the local solar time.
       Returns data aray of same shape (i.e. 2D lat long) """
    
    
    UTC_da = xr.DataArray(reference_time, 
                          coords=
                              {
                               'y': ds.y.data,
                               'x':ds.x.data
                               }, dims=['y','x'])
    
    output_array = UTC_da - UTC_da.x/15
    
    return  output_array


def absolute_datetime(ds,f):
    """ Read the date from the filename and combine with the UTC to create an absolute datetime.
        Assumes specific form of filename for string splitting """
        
    #Read date from string
    date_string =f.split('_')[-1].split('.')[0].replace('-','')
    date = pd.to_datetime(date_string, format='%Y%m%d')
    
    
    #Combine with UTC
    datetime = (date + (ds_MODIS['UTC']* 3600).astype("timedelta64[s]")).dt.round('H')
    return datetime




In [15]:
scratch = ds_MODIS

In [16]:
%%time

reference_time = 13.5
scratch['UTC'] = method_one(ds_MODIS,reference_time)
scratch['dateUTC'] = absolute_datetime(ds_MODIS,root+f)


CPU times: user 5.1 s, sys: 1.62 s, total: 6.72 s
Wall time: 6.71 s


In [18]:
%%time

reference_time = 13.5
method_one(ds_MODIS,reference_time)
#absolute_datetime(ds_MODIS,root+f)



CPU times: user 149 ms, sys: 118 ms, total: 267 ms
Wall time: 265 ms


<xarray.DataArray (y: 5400, x: 10800)>
array([[25.49888889, 25.49666667, 25.49444444, ...,  1.50555556,
         1.50333333,  1.50111111],
       [25.49888889, 25.49666667, 25.49444444, ...,  1.50555556,
         1.50333333,  1.50111111],
       [25.49888889, 25.49666667, 25.49444444, ...,  1.50555556,
         1.50333333,  1.50111111],
       ...,
       [25.49888889, 25.49666667, 25.49444444, ...,  1.50555556,
         1.50333333,  1.50111111],
       [25.49888889, 25.49666667, 25.49444444, ...,  1.50555556,
         1.50333333,  1.50111111],
       [25.49888889, 25.49666667, 25.49444444, ...,  1.50555556,
         1.50333333,  1.50111111]])
Coordinates:
  * y        (y) float64 89.98 89.95 89.92 89.88 ... -89.88 -89.92 -89.95 -89.98
  * x        (x) float64 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 179.9 180.0

# Method 2

In [7]:
def method_two(ds,reference_time):

    ref_data = np.ones((ds.sizes['y'], ds_MODIS.sizes['x'])) * reference_time #array of 
    new = ds_MODIS.LST.copy(data=ref_data)


    out = new - new.x/15

    return out

In [19]:
%%time

reference_time = 13.5
scratch['UTC'] = method_two(ds_MODIS,reference_time)
scratch['dateUTC'] = absolute_datetime(ds_MODIS,root+f)



CPU times: user 5.26 s, sys: 2.2 s, total: 7.46 s
Wall time: 7.45 s
